# Preparação de Dados

## Carregar o dataset

O código na célula abaixo importa os pacotes necessários para executar este notebook.


In [ ]:
import pandas as pd
import numpy as np

%matplotlib inline

O código na célula abaixo carrega o dataset e imprime as primeiras linhas do data frame.

In [ ]:
enem = pd.read_csv("../data/projeto_ENEM.csv")
enem.head(20)

## Tratamento dos valores ausentes

Existem 5767 linhas com município de nascimento ausente. Esse dado está relacionado com outros dados municipais como município de residência e município da aplicação da prova, que pela contemporaneidade devem servir melhor para predizer o desempenho do candidato. Portanto, decidimos eliminar as colunas relativas a esse dado.

Existem 121 mil linhas com dados da escola ausente, mas não podemos simplesmente eliminar essas colunas, tendo em vista que a escola é muito importante para predizer o desempenho do candidato. Desse registro, vamos manter somente a chave do registro, a coluna `CO_ESCOLA`, imputando-lhe o valor $0$ quando estiver ausente. Na seção sobre engenharia de features abaixo, mostraremos como iremos usar essa coluna para encontrar outras features mais úteis.

Existem 6 linhas com `NU_IDADE` ausente e 5631 linhas com `TP_ESTADO_CIVIL` ausente. Como esses dados têm alguma importância no desempenho do candidato e são poucas em relação ao total, decidimos eliminar todas as linhas que têm esses dados ausentes.

O código na célula abaixo faz as transformações descritas.

In [ ]:
enem.drop(
    [
        "CO_MUNICIPIO_NASCIMENTO",
        "NO_MUNICIPIO_NASCIMENTO",
        "CO_UF_NASCIMENTO",
        "SG_UF_NASCIMENTO",
        "TP_ENSINO",
        "CO_MUNICIPIO_ESC",
        "NO_MUNICIPIO_ESC",
        "CO_UF_ESC",
        "SG_UF_ESC",
        "TP_DEPENDENCIA_ADM_ESC",
        "TP_LOCALIZACAO_ESC",
        "TP_SIT_FUNC_ESC",
    ],
    axis=1,
    inplace=True,
)
enem.loc[enem["CO_ESCOLA"].isna(), "CO_ESCOLA"] = 0
enem = enem[enem["NU_IDADE"].notna() & enem["TP_ESTADO_CIVIL"].notna()]
enem.shape

### Transformar o tipo de dados das colunas

Existem três colunas nesse dataset que não têm o tipo correto como resultado dos valores ausentes.

O código na célula abaixo itera sobre a lista de colunas os definindo como inteiros.

In [ ]:
cols = ["NU_IDADE", "TP_ESTADO_CIVIL", "CO_ESCOLA"]
for col in cols:
    enem[col] = pd.to_numeric(enem[col], downcast="integer")
enem[cols].dtypes

O código a seguir cria as colunas dos labels, $0$ para o candidato *reprovado* na prova, que tirou nota menor do que 650, e $1$ para o candidato *aprovado* na prova, que tirou nota 650 ou mais. Também elimina as colunas com as notas numéricas.

In [ ]:
provas = ["CN", "CH", "LC", "MT"]
for prova in provas:
    nu_nota_prova = "NU_NOTA_" + prova
    enem["LB_APROVADO_" + prova] = np.where(enem[nu_nota_prova] < 650, 0, 1)
    enem.drop([nu_nota_prova], axis=1, inplace=True)
enem.head(20)

### Engenharia de Features
#### Agregando a idade
O código abaixo calcula, para cada idade, quantos candidatos estão inscritos e proporção de aprovados em cada prova e grava tudo no arquivo &ldquo;data/idades.csv&rdquo;.

In [ ]:
idades = enem.groupby(by="NU_IDADE", as_index=False).agg(
    {
        "NU_INSCRICAO": np.size,
        "LB_APROVADO_CN": np.sum,
        "LB_APROVADO_CH": np.sum,
        "LB_APROVADO_LC": np.sum,
        "LB_APROVADO_MT": np.sum,
    }
)
for prova in provas:
    idades["PC_APROVACAO_" + prova] = (
        idades["LB_APROVADO_" + prova] / idades["NU_INSCRICAO"]
    )
idades.to_csv("../data/idades.csv", index=False)
idades.head(20)

Podemos ver que a taxa de aprovação aumenta na faixa dos 15 à 20 anos, onde também estão a grande maioria dos candidatos, e decresce para os extremos. Portanto não é conveniente usar a idade diretamente para treinar o modelo, será melhor a transformar em um feature categórico dividindo em 25 faixas etárias:

| Faixa Etária |
|--------------|
| < 15 anos    |
| 15 anos      |
| 16 anos |
| 17 anos |
| 18 anos |
| 19 anos |
| 20 anos |
| 21 anos |
| 22 anos |
| 23 anos |
| 24 anos |
| 25 anos |
| 26 anos |
| 27 anos |
| 28 anos |
| 29 anos |
| 30 anos |
| 31&ndash;32 anos |
| 33&ndash;34 anos |
| 35&ndash;36 anos |
| 37&ndash;38 anos |
| 39&ndash;41 anos |
| 42&ndash;45 anos |
| 46&ndash;50 anos |
| >50 anos |

O código seguinte cria a coluna `TP_FAIXA_ETARIA`, que faz a agregação da idade explicada.

In [ ]:
criteria = [
    enem["NU_IDADE"] < 15,
    enem["NU_IDADE"] == 15,
    enem["NU_IDADE"] == 16,
    enem["NU_IDADE"] == 17,
    enem["NU_IDADE"] == 18,
    enem["NU_IDADE"] == 19,
    enem["NU_IDADE"] == 20,
    enem["NU_IDADE"] == 21,
    enem["NU_IDADE"] == 22,
    enem["NU_IDADE"] == 23,
    enem["NU_IDADE"] == 24,
    enem["NU_IDADE"] == 25,
    enem["NU_IDADE"] == 26,
    enem["NU_IDADE"] == 27,
    enem["NU_IDADE"] == 28,
    enem["NU_IDADE"] == 29,
    enem["NU_IDADE"] == 30,
    enem["NU_IDADE"].between(31,32),
    enem["NU_IDADE"].between(33,34),
    enem["NU_IDADE"].between(35,36),
    enem["NU_IDADE"].between(37,38),
    enem["NU_IDADE"].between(39,41),
    enem["NU_IDADE"].between(42,45),
    enem["NU_IDADE"].between(46,50),
    enem["NU_IDADE"] > 50,
]
values = [
    "< 15 anos",
    "15 anos",
    "16 anos",
    "17 anos",
    "18 anos",
    "19 anos",
    "20 anos",
    "21 anos",
    "22 anos",
    "23 anos",
    "24 anos",
    "25 anos",
    "26 anos",
    "27 anos",
    "28 anos",
    "29 anos",
    "30 anos",
    "31--32 anos",
    "33--34 anos",
    "35--36 anos",
    "37--38 anos",
    "39--41 anos",
    "42--45 anos",
    "46--50 anos",
    ">50 anos",
]
enem["TP_FAIXA_ETARIA"] = np.select(criteria, values)
enem.head(20)

###Uso do municipio de residência e da escola

Notamos a existência de dois dados que são importantes para predizer o desempenho do estudante: o município de residência e a escola do candidato, porém, como existem milhares de municípios e escolas, não podemos usar esses dados crus para treinar os modelos, encontramos outros atributos do município e da escola que tenham poder preditivo como mostramos a seguir.

Para cada municipio e escola, calculamos a proporção de candidatos aprovados em cada prova e utilizaremos esses dados como features no treinamento dos modelos.

A partir do arquivo inteiro de microdados do Enem, geramos dois conjuntos de dados: um para os municípios, `dados/municipios.csv`, e outro para as escolas, `dados/escolas.csv`. Como o número de registro dos microdados do Enem é grande demais para ser tratado da maneira tradicional usando Pandas, tivemos que desenvolver um programa específico, cujo o código Python está no arquivo `read_microdados.py`. Em particular, o programa tem tempo de execução $O(n)$ e uso de memória $O(e + m)$ onde $n$ é o número de registros de microdados do Enem, $e$ é o número de escolas e $m$ o número de municípios.

O código abaixo carrega o conjunto de dados relativos aos municípios.

In [ ]:
municipios = pd.read_csv("../data/municipios.csv")
municipios.head(20)

E o seguinte carrega o conjunto de dados relativos às escolas.

In [ ]:
escolas = pd.read_csv("../data/escolas.csv")
escolas.head(20)

Agora devemos fazer uma join das das duas tabelas com nossa tabela principal através do código abaixo.

In [ ]:
enem = pd.merge(
    enem, municipios, left_on="CO_MUNICIPIO_RESIDENCIA", right_on="CO_MUNICIPIO"
)
enem = pd.merge(enem, escolas, on="CO_ESCOLA")
enem.shape

Assim chegamos ao dataset final com 171 mil linhas e 125 colunas.

Vamos gravar o dataset para um arquivo csv.

In [ ]:
enem.to_csv('../data/enem_preparado.csv', index = False, header = True)
